##### Basic Model monitoring

In [2]:

import os
import pickle
import pandas as pd
import mlflow
import mlflow.pyfunc
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, ColumnQuantileMetric, DatasetDriftMetric, DatasetMissingValuesMetric,\
    DatasetCorrelationsMetric

from joblib import load, dump
from tqdm import tqdm

In [3]:
def get_model_from_registry(version_no = None):
    mlflow.set_tracking_uri('sqlite:///mlflow.db')
    model_name = 'best-DnD-RFModel'
    if version_no is not None:
        model_version = version_no
    else:
        client = MlflowClient(tracking_uri = "sqlite:///mlflow.db")
        # Below will return warning as get_latest_versions is deprecated in mlflow since 2.9.0
        meta_data = client.get_latest_versions(model_name, stages = ["None"])
        model_version = meta_data[0].version
    try: 
        model = mlflow.pyfunc.load_model(model_uri = f"models:/{model_name}/{model_version}")
    except OSError as ose:
        metadata_source = meta_data[0].source
        idx = metadata_source.index('/mlruns')
        path = os.getcwd() + metadata_source[idx:] + '/model.pkl'
        with open(path, 'rb') as modelFile:
            model = pickle.load(modelFile)
    return model